In [13]:
import numpy as np

## Part 1

# Batch mode training using least squares - supervised learning of network weights. 


- Implement a radial basis function network from scratch. 

- The network will be used to approximate sin(2x) and square(2x) functions




In [14]:
## Support functions for the evaluation

def getTrainSet(func = 'sin2x', stepSize = 0.1):
    ## Returns an 2 x N array of a training set
    # Row 0 = inputs and row 1 = targets¨
    # stepSize is taken as input, range is fixed to 0 --> 2pi. 
    N = int(np.floor(2 * np.pi/stepSize)) #Number of datapoints
    
    train = np.zeros((2,N))
    inputs = np.arange(0, N)
    np.random.shuffle(inputs)
    
    for step, i in enumerate(inputs):
        train[0, i] = step*stepSize # Input: will be for example 0, 0.1, 0.2 .. 2pi etc.. 
        if (func == 'sin2x'): 
            train[1, i] = np.sin(2*step*stepSize) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..
        elif (func == 'step2x'): 
            train[1, i] = np.sign(np.sin(2*step*stepSize)) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..

    return train.T

def getTestSet(func = 'sin2x', stepSize = 0.1):
    ## Returns an 2 x N array of a training set
    # Row 0 = inputs and row 1 = targets¨
    # stepSize is taken as input, range is fixed to 0 --> 2pi. 
    N = int(np.floor(2 * np.pi/stepSize)) #Number of datapoints
    
    test = np.zeros((2,N))

    for step in range(N):
        test[0, step] = step*stepSize+0.05 # Input: will be for example 0.05, 0.15, 0.25 .. 2pi´0.05 etc.. 
        if (func == 'sin2x'): 
            test[1, step] = np.sin(2*step*stepSize+0.05) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..
        elif (func == 'step2x'): 
            test[1, step] = np.sign(np.sin(2*step*stepSize+0.05)) # Target: for example sin(2*0), sin(2*0.1) .. sin(2*2pi) etc..

    return test.T

In [15]:
class RBF:
    def __init__(self, n = 12, variance = 0.1, maxinput = 6.28):
        self.n = n # the number of nodes
        self.variance = 0.1 ## Same variance for all nodes
        # self.units = np.random.rand(1, n) * maxinput # random unit position in the input space
        # self.units = np.array([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]) # unit positions 
        
        self.units = np.arange(0, maxinput, maxinput/(n-1))
        self.units = np.append(self.units, maxinput)
        self.n = self.units.shape[0]
        self.w = np.random.rand(1,n)
    
    def error(self, f_approx, f):
        if not (f_approx.shape == f.shape): 
            raise Exception('f_approx and f shapes mismatch. f_approx: {}, f: {} '.format(f_approx.shape, f.shape))

        # error = (phi(x) - target)^2
        return np.average(np.abs(f_approx - f))
    
    def predict(self, inp):
        # takes an input inp and returns predictions
        # do f^ = phi(x) * w.T
        x = inp.dot(np.ones((1, self.n))) # inp x 1 * 1 x n --> inp x n
        
        # m x n * n x 1 --> m x 1
        # print("m x n:")
        # print(x.shape)
        print("n x 1:")
        print(self.w.shape)
        f_approx = self.phi_matrix(x).dot(self.w)
        # print("output shape: " + str(f_approx.shape))
        return f_approx

    
    def fit(self, inp, f):
        # x is the input. Shape: inputs x 1
        # f is the true value of the function (aka the target), same shape
        if not (inp.shape == f.shape): 
            raise Exception('inp and f shapes mismatch. inp: {}, f: {} '.format(x.shape, f.shape))
        
        # print("input shape (m x 1):" + str(inp.shape))
        # print("target shape (m x 1):" + str(f.shape))

        # We want x to be a matrix with shape: inputs x neurons
        x = inp.dot(np.ones((1, self.n))) # inp x 1 * 1 x n --> inp x n
        
        # print("creted x with shape " + str(x.shape))
        
        # get phi(x)
        self.phi_x = self.phi_matrix(x)
        # print(self.phi_x.shape)

        # we obtain the w that minimizes the error by solving: 
        # phi(x).T * phi(x) * w = phi(x).T * f
        # --> w = (phi(x).T * phi(x))^-1 * phi(x).T * f
        # print(f.shape)
        
        self.w = np.linalg.inv(self.phi_x.T.dot(self.phi_x)).dot(self.phi_x.T).dot(f)
        
        print('Model fitted')
    
    def phi(self, x, i): 
        return np.exp(-(np.square(x-i))/(2*np.square(self.variance)))
    
    def phi_matrix(self, x):
        # number of inputs (m) (rows) x self.n (n) (columns)
        print("phi_matrix input size:" + str(x.shape))
        # this is a slow and stupid way of computing phi(x)
        for m in range(x.shape[0]): # m
            for n in range(x.shape[1]): # n
                res = self.phi(x = x[m, n], i = self.units[n])
        #        print("phi({}, {}): {}".format(x[row, col], self.units[0, row], res))
                x[m, n] = res

        return x
    
    

In [24]:
import matplotlib.pyplot as plt
def test(n):     
    model = RBF(n, variance = 0.1)
    
    function = 'sin2x'
    
    # Train model
    trainset = getTrainSet(function)
    # print(function + ' train set example: ')
    # print(trainset[0:5, :])
    # print("Units: ")
    # print(model.units)
    model.fit(trainset[:, 0][np.newaxis,:].T, trainset[:, 1][np.newaxis,:].T)
    
    
    ## Test Model
    
    # Get test set
    testset = getTestSet(function)
    # Make predictions
    pred = model.predict(testset[:, 0][np.newaxis,:].T)
    # test error
    e = model.error(pred, testset[:, 1][np.newaxis,:].T)
    # 
    # print("predictions")
    # print(pred[0:5, :])
    # print("target")
    # print(testset[0:5, :])
    # 
    # print("Absolute residual error: {}".format(e))
    # 
    return e

# nvars = np.array([10, 20, 30, 40, 50, 60, 61, 62, 63])
# fvars = np.ones(nvars.shape)
# for i, n in enumerate(nvars):
#     fvars[i] = test(n)
#  

print("{} units evenly distributed. Res. Err: {}".format(6, round(test(6), 3)))
print("{} units evenly distributed. Res. Err: {}".format(12, round(test(12), 3)))
print("{} units evenly distributed. Res. Err: {}".format(20, round(test(20), 3)))
print("{} units evenly distributed. Res. Err: {}".format(30, round(test(30), 3)))
print("{} units evenly distributed. Res. Err: {}".format(60, round(test(60), 3)))

phi_matrix input size:(62, 6)
Model fitted
n x 1:
(6, 1)
phi_matrix input size:(62, 6)
6 units evenly distributed. Res. Err: 0.517
phi_matrix input size:(62, 12)
Model fitted
n x 1:
(12, 1)
phi_matrix input size:(62, 12)
12 units evenly distributed. Res. Err: 0.368
phi_matrix input size:(62, 20)
Model fitted
n x 1:
(20, 1)
phi_matrix input size:(62, 20)
20 units evenly distributed. Res. Err: 0.152
phi_matrix input size:(62, 30)
Model fitted
n x 1:
(30, 1)
phi_matrix input size:(62, 30)
30 units evenly distributed. Res. Err: 0.036
phi_matrix input size:(62, 60)
Model fitted
n x 1:
(60, 1)
phi_matrix input size:(62, 60)
60 units evenly distributed. Res. Err: 0.035


### Minimizing the residual error

With 12 units evenly distributed, variance = 0.1, the Absolute residual error was: 0.32240
the error did not change significantly with variance 0.3 or 0.01


With 20 units evenly distributed, variance = 0.1, the Absolute residual error was: 0.131492


Adding a node in the very end of the interval reduced it somewhat to 0.12314

30 units: 0.03243

40 units: 0.0313 


#### Questions:

- What is the lower bound for the number of training examples, N?
N must be larger than the number of neurons, n. 

- What happens with the error if N = n? Why?
The error starts increasing. This is because the system is overdetermined. 
The model can only reach points lying in some fixed plane and can never exactly match y^ if it lies somewhere outside the plane

- Under what conditions, if any, does (4) have a solution in this case?
Yes, if some equation occurs several times in the system, or if some equations are linear combinations of the others. 
